<a href="https://colab.research.google.com/github/kellykryoung/AI_Class/blob/main/kaggle_catdog_CNN_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# data : https://www.kaggle.com/datasets/tongpython/cat-and-dog/?select=test_set
# code reference: https://wikidocs.net/63565

In [10]:
import os
from tqdm import tqdm

import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torch import nn, optim
from torchvision import datasets

from PIL import Image
from torchvision.transforms import ToPILImage
import torchvision.transforms as T

import matplotlib.pyplot as plt
import numpy as np

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# 랜덤시드 고정
torch.manual_seed(777)

# GPU 사용 가능일 경우 랜덤 시드 고정
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

# hyper-parameters
learning_rate = 0.001
training_epochs =15
batch_size = 100


### 데이터

In [4]:
!unzip '/content/training_set.zip'

unzip:  cannot find or open /content/training_set.zip, /content/training_set.zip.zip or /content/training_set.zip.ZIP.


In [5]:
!unzip '/content/test_set.zip'

unzip:  cannot find or open /content/test_set.zip, /content/test_set.zip.zip or /content/test_set.zip.ZIP.


In [6]:
class CustomDataset(Dataset):
    def __init__(self, root_dir, transform=None):

        self.root_dir= root_dir
        self.transform = transform
        self.classes = os.listdir(root_dir)

        self.data = []

        for label in range(len(self.classes)): # 라벨수
            class_folder = os.path.join(root_dir, self.classes[label])
            for filename in os.listdir(class_folder):
                img_path = os.path.join(class_folder, filename)
                self.data.append((img_path, label)) # tuple로 append

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path, label = self.data[idx]
        image = Image.open(img_path)
        if self.transform:
            image = self.transform(image)
        return image, label



In [8]:
data_dir = "." #현재 작업 디렉토리로 설정
batch_size = 32

transform = T.Compose([T.Resize((224,224)),
                       T.RandomRotation(5),
                       T.RandomHorizontalFlip(),
                           T.ToTensor(),
                           T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
                           ])

# 01_dataset을 directory로 정의하기 (class 이용)
train_dataset = CustomDataset(os.path.join(data_dir, 'training_set'), transform=transform)
test_dataset = CustomDataset(os.path.join(data_dir, 'test_set'), transform=transform)

# 02_dataloader
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


FileNotFoundError: ignored

### 모델

In [ ]:
class CNN(torch.nn.Module):

    def __init__(self):
        super(CNN, self).__init__()
        # 첫번째층
        # ImgIn shape=(?, 224, 224, 3)
        #    Conv     -> (?, 224, 224, 32)
        #    Pool     -> (?, 112, 112, 32)
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
            torch.nn.BatchNorm2d(),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))

        # 두번째층
        # ImgIn shape=(?, 112, 112, 32)
        #    Conv      ->(?, 112, 112, 64)
        #    Pool      ->(?, 56, 56, 64)
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            torch.nn.BatchNorm2d(),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))

        # 세번째층
        # ImgIn shape=(?, 56, 56, 64)
        #    Conv      ->(?, 56, 56, 128)
        #    Pool      ->(?, 28, 28, 128)

        self.layer3 = torch.nn.Sequential(
            torch.nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            torch.nn.BatchNorm2d(),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))

        # 네번째층
        # ImgIn shape=(?, 28, 28, 128)
        #    Conv      ->(?, 28, 28, 256)
        #    Pool      ->(?, 14, 14, 256)

        self.layer4 = torch.nn.Sequential(
            torch.nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
            torch.nn.BatchNorm2d(),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))

        # 다섯번째층
        # ImgIn shape=(?, 14, 14, 256)
        #    Conv      ->(?, 14, 14, 128)
        #    Pool      ->(?, 7, 7, 128)
        self.layer5 = torch.nn.Sequential(
            torch.nn.Conv2d(256, 128, kernel_size=3, stride=1, padding=1),
            torch.nn.BatchNorm2d(),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))


        # 전결합층 7x7x128 inputs -> 2 outputs
        self.fc = torch.nn.Linear(7 * 7 * 128, 2, bias=True)

        # 전결합층 한정으로 가중치 초기화
        torch.nn.init.xavier_uniform_(self.fc.weight)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.layer5(out)
        out = out.view(out.size(0), -1)   # 전결합층을 위해서 Flatten
        out = self.fc(out)
        return out

In [ ]:
# model class 호출
cnn_model = CNN().to(device)

### 옵티마이저

In [ ]:
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.SGD(cnn_model.parameters(), lr=0.001, momentum=0.9)

### 포문 훈련

In [ ]:
total_batch = len(train_loader)
print('총 배치의 수 : {}'.format(total_batch))

In [ ]:
for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in tqdm(train_loader):
        X = X.to(device) # loader할 때마다 .to(deivce)
        Y = Y.to(device)

        hypothesis = cnn_model(X)
        cost = criterion(hypothesis, Y)

        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch

    print(f'[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

torch.save(cnn_model.state_dict(), 'cnn_model.pth')

### 평가

In [ ]:
cnn_model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)

        outputs = cnn_model(images)
        _, predicted = torch.max(outputs, 1)

        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'Accuracy on test set: {accuracy}%')